In [1]:
from tensorflow.keras.applications import EfficientNetB0

In [2]:
model_base = EfficientNetB0(weights='imagenet', 
                            input_shape=(224,224,3),
                            include_top=False)

model_base.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
base_dir = '../data/cat_dog_full'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

datagen = ImageDataGenerator()
batch_size = 50

def extract_feature(directory, sample_count):
    features = np.zeros(shape=(sample_count,7,7,1280))
    labels = np.zeros(shape=(sample_count,))
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='binary'
    )
    
    i = 0
    for x_data_batch, t_data_batch in generator:
        feature_batch = model_base.predict(x_data_batch)
        features[i*batch_size:(i+1)*batch_size] = feature_batch
        labels[i*batch_size:(i+1)*batch_size] = t_data_batch
        i+=1
        if i * batch_size >= sample_count:
            break
    
    return features, labels

In [7]:
trian_features, train_labels = extract_feature(train_dir, 14000)
validation_features, validation_labels = extract_feature(validation_dir, 6000)

Found 14000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

trian_features = np.reshape(trian_features, (14000,7*7*1280))
validation_features = np.reshape(validation_features, (6000,7*7*1280))

In [9]:
model = Sequential()
model.add(Dense(units=1024, activation='relu', input_shape=(7*7*1280,)))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))

model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(x=trian_features, y=train_labels, epochs=30,
                    validation_data=(validation_features, validation_labels))

Epoch 1/30
438/438 [==============================] - 24s 54ms/step - loss: 0.0666 - accuracy: 0.9764 - val_loss: 0.0327 - val_accuracy: 0.9875
Epoch 2/30
438/438 [==============================] - 9s 21ms/step - loss: 0.0090 - accuracy: 0.9977 - val_loss: 0.0382 - val_accuracy: 0.9862
Epoch 3/30
438/438 [==============================] - 9s 21ms/step - loss: 0.0035 - accuracy: 0.9994 - val_loss: 0.0375 - val_accuracy: 0.9875
Epoch 4/30
438/438 [==============================] - 9s 21ms/step - loss: 0.0013 - accuracy: 0.9999 - val_loss: 0.0406 - val_accuracy: 0.9882
Epoch 5/30
438/438 [==============================] - 9s 21ms/step - loss: 6.2508e-04 - accuracy: 0.9999 - val_loss: 0.0410 - val_accuracy: 0.9873
Epoch 6/30
438/438 [==============================] - 9s 20ms/step - loss: 3.7309e-04 - accuracy: 1.0000 - val_loss: 0.0436 - val_accuracy: 0.9878
Epoch 7/30
438/438 [==============================] - 9s 20ms/step - loss: 2.0680e-04 - accuracy: 1.0000 - val_loss: 0.0458 - val_acc